# Setup

In [1]:
from IPython.display import clear_output
from random import randint
from copy import deepcopy
from itertools import combinations
from random import randint
from time import sleep
from typing import Tuple, List
import numpy as np
import random as rnd

# Funciones de lógica

In [2]:
class Descriptor :

    '''
    Codifica un descriptor de N argumentos mediante un solo caracter
    Input:  args_lista, lista con el total de opciones para cada
                     argumento del descriptor
            chrInit, entero que determina el comienzo de la codificación chr()
    Output: str de longitud 1
    '''

    def __init__ (self,args_lista,chrInit=256) :
        self.args_lista = args_lista
        assert(len(args_lista) > 0), "Debe haber por lo menos un argumento"
        self.chrInit = chrInit
        self.rango = [chrInit, chrInit + np.prod(self.args_lista)]

    def check_lista_valores(self,lista_valores) :
        for i, v in enumerate(lista_valores) :
            assert(v >= 0), "Valores deben ser no negativos"
            assert(v < self.args_lista[i]), f"Valor debe ser menor a máximo {self.args_lista[i]}"

    def codifica(self,lista_valores) :
        self.check_lista_valores(lista_valores)
        cod = lista_valores[0]
        n_columnas = 1
        for i in range(0, len(lista_valores) - 1) :
            n_columnas = n_columnas * self.args_lista[i]
            cod = n_columnas * lista_valores[i+1] + cod
        return cod

    def decodifica(self,n) :
        decods = []
        if len(self.args_lista) > 1:
            for i in range(0, len(self.args_lista) - 1) :
                n_columnas = np.prod(self.args_lista[:-(i+1)])
                decods.insert(0, int(n / n_columnas))
                n = n % n_columnas
        decods.insert(0, n % self.args_lista[0])
        return decods

    def P(self,lista_valores) :
        codigo = self.codifica(lista_valores)
        return chr(self.chrInit+codigo)

    def inv(self,codigo) :
        n = ord(codigo)-self.chrInit
        return self.decodifica(n)

    def escribir(self, literal,informacion):
        if '-' in literal:
            atomo = literal[1:]
            neg = ' no'
        else:
            atomo = literal
            neg = ''
        x, y, o = self.inv(atomo)
        if o in [0, 2, 4]:
            actitud = ' sabe que'
            return f"El agente{actitud}{neg} {informacion[o]} la casilla ({x},{y})"
        elif o in [1, 3]:
            actitud = ' siente'
            return f"El agente{neg}{actitud} {informacion[o]} la casilla ({x},{y})"


class ClausulaDefinida :
    '''
    Implementación de las cláusulas definidas
    Input: clausula, que es una cadena de la forma p1 Y ... Y pn > q
    '''

    def __init__(self, clausula) :
        self.nombre = clausula
        indice_conectivo = clausula.find('>')
        if indice_conectivo > 0:
            cuerpo = clausula[:indice_conectivo].split('Y')
            cabeza = clausula[indice_conectivo + 1:]
        else:
            cuerpo = clausula.split('Y')
            cabeza = ''
        self.cuerpo = cuerpo
        self.cabeza = cabeza


class LPQuery:

    '''
    Implementación de una base de conocimiento.
    Input:  base_conocimiento_lista, que es una lista de cláusulas definidas
                de la forma p1 Y ... Y pn > q
            cods, un objeto de clase Descriptor
    '''

    def __init__(self, base_conocimiento_lista) :
        self.datos = []
        self.reglas = []
        self.atomos = []
        for formula in base_conocimiento_lista:
            self.TELL(formula)

    def __str__(self) :
        cadena = 'Datos:\n'
        for dato in self.datos:
            cadena += dato + '\n'
        cadena += '\nReglas:\n'
        for regla in self.reglas:
            cadena += regla.nombre + '\n'
        return cadena

    def reglas_aplicables(self, head):
        return [r for r in self.reglas if r.cabeza == head]

    def test_objetivo(self, literal):
        return literal in self.datos

    def TELL(self, formula):
        indice_conectivo = formula.find('>')
        if indice_conectivo > 0:
            clausula = ClausulaDefinida(formula)
            self.reglas.append(clausula)
            for a in clausula.cuerpo:
            	if '-' in a:
            		atomo = a[1:]
            	else:
                	atomo = a
            	if atomo not in self.atomos:
            		self.atomos.append(atomo)
            if '-' in clausula.cabeza:
            	atomo = clausula.cabeza[1:]
            else:
            	atomo = clausula.cabeza
            if atomo not in self.atomos:
            	self.atomos.append(atomo)
        else:
            for literal in formula.split('Y'):
                if literal not in self.datos:
                    self.datos.append(literal)
                    if '-' in literal:
                    	atomo = literal[1:]
                    else:
                    	atomo = literal
                    if literal not in self.atomos:
                    	self.atomos.append(atomo)

def pl_fc_entails(base, q) :
    count = {}
    for regla in base.reglas:
        count[regla.nombre] = len(regla.cuerpo)
    inferred = dict(zip(base.atomos, [False]*len(base.atomos)))
    queue = deepcopy(base.datos)
    while len(queue) > 0:
        p = queue.pop()
        if p == q:
            return True
        elif inferred[p] == False:
            inferred[p] = True
            for regla in base.reglas:
                if p in regla.cuerpo:
                    count[regla.nombre] -= 1
                    if count[regla.nombre] == 0:
                        queue.append(regla.cabeza)
    return False

def and_or_graph_search(objetivo, base):
    return or_search(objetivo, base, [])

def or_search(head, base, camino):
    if base.test_objetivo(head):
        return 'success'
    elif head in camino:
        return 'failure'
    reglas = base.reglas_aplicables(head)
    if not reglas:
        return 'failure'
    for regla in reglas:
        plan = and_search(regla.cuerpo, base, [head] + camino)
        if plan != 'failure':
            return 'success'
    return 'failure'

def and_search(literales, base, camino):
    for literal in literales:
        plan = or_search(literal, base, camino)
        if plan == 'failure':
            return 'failure'
    return 'success'

def ASK(objetivo, valor, base):
    ask = and_or_graph_search(objetivo, base)
    return (ask == valor)


# Funciones adicionales para MineSweeper

In [3]:
def truncar(val: int, lim: int):
    """
    Función que delimita las coordenadas a la designada por el jugador.
    --------------------------------------------------------------------------
    Keyword Arguments:
    val:int -- valor que se desea truncar.
    lim:int -- límite con el cual se desea truncar.
    --------------------------------------------------------------------------
    """

    if val < 0:
        return 0
    elif val > lim - 1:
        return lim - 1
    else:
        return val


def adyacentes(coord: Tuple[int, int], dims: Tuple[int, int]) -> List[Tuple[int, int]]:
    """
    Funcion que retorna todos los puntos adyacentes a las coordenadas 'coord',
    en un espacio de dimensión 'dims'.
    --------------------------------------------------------------------------
    Keyword Arguments:
    coord:Tuple[int, int]   -- Tupla con valores 'x' y 'y' de la coordenada
                               deseada.
    dims:Tuple[int, int]    -- Dimensiones del tablero en el que se está
                               jugando.
    --------------------------------------------------------------------------
    Posibles casillas adyacentes para la casilla marcada como 'O':
    XXX
    XOX
    XXX
    """
    x, y = coord
    lx, ly = dims
    ady = {
        (truncar(x - 1, lx), truncar(y - 1, ly)),
        (x, truncar(y - 1, ly)),
        (truncar(x + 1, lx), truncar(y - 1, ly)),
        (truncar(x - 1, lx), y),
        (truncar(x + 1, lx), y),
        (truncar(x - 1, lx), truncar(y + 1, ly)),
        (x, truncar(y + 1, ly)),
        (truncar(x + 1, lx), truncar(y + 1, ly)),
    }
    return sorted(list(ady))


def codeStrMina(val: int) -> str:
    """
    Esta función cumple con la función de traducir los caracteres utilizados
    para caracterizar los estados de las casillas,
    --------------------------------------------------------------------------
    """
    if val >= 0 and val < 9:
        return f"{val}    "
    elif val == 9:
        return "💣"
    elif val == 10:
        return "⚑"
    elif val == 11:
        return "■"


# Interfaz MineSweeper

In [4]:
class MineSweeper:
    """Esta clase funciona como interfaz para el desarrollo de experimentos
    relacionados con el juego de buscaminas."""

    def __init__(self, x:int, y:int, m:int, d:Descriptor):

        self.desc = d

        # Para correr las iteraciones y tener un método para parar.
        self.jugando = True

        # 'x' cantidad de columnas y 'y' cantidad de filas.
        self.x = x
        self.y = y

        # Estado del juego:
        # 0 - 8 : Cantidad de minas adyacentes
        #     9 : Mina confirmada
        #    10 : Mina ha sido marcada
        #    11 : Casilla desconocida

        self.display = np.full((x, y), 11)
        self.background = np.zeros((x, y))

        # Minas:
        self.minas_num = m
        self.minas_mar = 0
        self.minas_ubi = []

        # Marcas
        self.marcas = self.minas_num

        # Ubicar minas y num. adyacentes.
        for m in range(self.minas_num):
            # Ubica aleatoriamente cada una de las minas
            mx, my = (rnd.randint(0, x - 1), rnd.randint(0, y - 1))

            # Procura que las minas esten ubicadas en distintas posiciones.
            while (mx, my) in self.minas_ubi:
                mx, my = (rnd.randint(0, x - 1), rnd.randint(0, y - 1))

            # Pone la mina en el mapa vacio
            self.background[mx, my] = 10
            self.minas_ubi.append((mx, my))
            adys = adyacentes((mx, my), (x, y))

            # Segun las cada una de las minas modificar las adyacentes.
            for coord in adys:
                if self.background[coord] != 10:
                    self.background[coord] += 1


    def pintar_frente(self) -> None:
        """
        Pinta bonito el estado actual del juego (self.display).
        --------------------------------------------------------------------------
        """
        s = "$   "
        for i in range(self.x):
            s += f"{i:<5}"
        s += "\n"
        for i in range(self.x):
            s += f"{i:<4}"
            for j in range(self.y):
                s += f"{codeStrMina(self.display[(i,j)]):<4}"
            s += "\n"

        print(s)


    def pintar_atras(self) -> None:
        """
        Pinta bonito como está conformado el juego (self.background).
        --------------------------------------------------------------------------
        """
        s = "$   "
        for i in range(self.x):
            s += f"{i:<5}"
        s += "\n"
        for i in range(self.x):
            s += f"{i:<4}"
            for j in range(self.y):
                s += f"{codeStrMina(self.background[(i,j)]):<4}"
            s += "\n"

        print(s)


    def transicion(self, target: Tuple[int, int], accion: int = 0):
        """
        Según una posición y una acción, cambia el estado del juego
        (self.display).
        --------------------------------------------------------------------------
        Keyword Arguments:
        target:Tuple[int,int]   -- La coordenada que va a ser afectada.
        accion:int (default 0)  -- Depende de la acción suministrada:
                                    0 : Revelar lo que hay en el self.background.
                                    1 : Marcar mina.
        --------------------------------------------------------------------------
        """

        if self.jugando:

            if accion == 0:
                estado = self.background[target]
                self.display[target] = estado

                # Si lo que revela es una mina
                if estado == 9:
                    print("BOOOOOOM 💣")
                    self.jugando = False

            elif accion == 1:
                if self.marcas > 0:
                    self.estado[target] = 10
                else:
                    print("Ups... no hay más banderas 😞")
                    self.jugando = False

        else:
            print("WAT!... Pero si no estas jugando.")

        return self.background


    def objetivo(self) -> bool:
        """
        Función que revisa si todas las minas han sido marcadas, cuando todas lo
        han sido, se gana (termina) el juego.
        --------------------------------------------------------------------------
        """
        for m in self.minas_ubi:
            if self.display[m] != 10:
                return False
        return True

    def estado(self, target: Tuple[int, int]) -> int:
        """
        Retorna el estado en el self.display.
        --------------------------------------------------------------------------
        Keyword Arguments:
        target:Tuple[int,int]   -- La coordenada que va a ser afectada.
        --------------------------------------------------------------------------
        """
        return self.display[target]


# Definición de reglas

In [5]:
def formulas_unicidad(J: MineSweeper) -> List[str]:
    """
    Retorna formulas relacionadas con la unicidad de la casilla, e.i. si ya
    tiene una propiedad, ya no puede tener otra.
    --------------------------------------------------------------------------
    Keyword Arguments:
    J:MineSweeper -- Juego actual.
    --------------------------------------------------------------------------
    """
    formulas = []
    
    for xi in range(J.x):
        for yi in range(J.y):
            for i in ([ i for i in range(9) ] + [11]):
                for j in ([ i for i in range(9) ] + [11]):
                    if j != i:
                        formulas += [f"{J.desc.P([xi,yi,i])}>-{J.desc.P([xi,yi,j])}"]
                        
    return list(set(formulas))
  
"""    
    return list(
        set(
            [
                f"{J.desc.P([xi,yi,i])}>-{J.desc.P([xi,yi,j])}"
                for xi in range(J.x)
                for yi in range(J.y)
                for i in [ i for i in range(9) ] + [11]
                for j in [ i for i in range(9) ] + [11]
                if j != i
            ]
        )
    )
"""

'    \n    return list(\n        set(\n            [\n                f"{J.desc.P([xi,yi,i])}>-{J.desc.P([xi,yi,j])}"\n                for xi in range(J.x)\n                for yi in range(J.y)\n                for i in [ i for i in range(9) ] + [11]\n                for j in [ i for i in range(9) ] + [11]\n                if j != i\n            ]\n        )\n    )\n'

In [6]:
def formulas_perim_no_minas(J: MineSweeper) -> List[str]:
    """
    Esta función retorna todas las formulas que pueden ayudar a saber donde no
    hay minas.
    --------------------------------------------------------------------------
    Keyword Arguments:
    J:MineSweeper -- Juego actual.
    --------------------------------------------------------------------------
    """
    formulas = []

    for x in range(J.x):
        for y in range(J.y):
            
            Adyacentes = adyacentes((x, y), (J.x, J.y))
            # Caso en el que la posición no tiene minas adyacentes.
            formulas += [
                f"{J.desc.P([x,y,0])}>-{J.desc.P([ad[0],ad[1],9])}" for ad in Adyacentes
            ]

            # Caso en los que tiene de 1-8 minas adyacentes 💣
            for caso in range(1, 9):
                # Evalua todas las posibles formas en las que las minas podrian
                # estar ubicadas, teniendo en cuenta la cantidad de adyacentes.
                combinaciones = combinations(Adyacentes, caso)

                for posibilidad in combinaciones:

                    # Todos los posibles casos en donde no hay bombas a razón
                    # del caso actualmente a consideración.
                    a = [
                        f"Y-{J.desc.P([p[0],p[1],9])}"
                        for p in Adyacentes
                        if p not in posibilidad
                    ]
                    # Caso actualmente a consideración.
                    p = f"{J.desc.P([x,y,caso])}{''.join(a)}"
                    # Posibles consecuencias.
                    q = [f"{J.desc.P([p[0],p[1],9])}" for p in posibilidad]
                    # for all q_i in q : p -> q
                    formulas += [f"{p}>{qi}" for qi in q]

    return list(set(formulas))

In [7]:
def formulas_perim_minas(J: MineSweeper) -> List[str]:
    """
    Esta función retorna todas las formulas que pueden ayudar a saber donde hay
    minas.
    --------------------------------------------------------------------------
    Keyword Arguments:
    J:MineSweeper -- Juego actual.
    --------------------------------------------------------------------------
    """
    formulas = []

    for x in range(J.x):
        for y in range(J.y):
            Adyacentes = adyacentes((x, y), (J.x, J.y))

            # Caso en los que tiene de 1-8 minas adyacentes 💣
            for caso in range(1, 9):
                # Evalua todas las posibles formas en las que las minas podrian
                # estar ubicadas, teniendo en cuenta la cantidad de adyacentes.
                combinaciones = combinations(Adyacentes, caso)
                for posibilidad in combinaciones:

                    # Todos los posibles estados en los que puede estar de forma
                    # conjunta al caso actualmente a consideración.
                    a = [f"Y{J.desc.P([p[0],p[1],9])}" for p in posibilidad]
                    # Caso actualmente a consideración.
                    p = f"{J.desc.P([x,y,caso])}{''.join(a)}"
                    # Consecuencias descartadas.
                    q = [
                        f"{J.desc.P([p[0],p[1],9])}"
                        for p in Adyacentes
                        if p not in posibilidad
                    ]
                    # for all q_i in q : p -> q
                    formulas += [f"{p}>{qi}" for qi in q]

    return list(set(formulas))

# Función para jugar

In [8]:
def GameOn(J: MineSweeper, B: LPQuery, P: bool = True) -> MineSweeper:
    """
    Corre el MineSweeper junto con la LPQuery hasta obtener una victoria o una
    derrota.
    --------------------------------------------------------------------------
    Keyword Arguments:
    J:MineSweeper   -- Juego actual.
    B:LPQuery       -- Base de conocimiento.
    P:bool          -- Si es verdadera, imprime todo. De ser falsa, solo retorna
                       resultados.
    --------------------------------------------------------------------------
    """
    # Comienza en esquina pues en otras posiciones retorna tanta información que
    # deja de se útil.
    x, y = (0, 0)
    J.transicion((x, y))
    if P:
        clear_output(wait=True)
        J.pintar_frente()

    B.TELL(J.desc.P([x, y, J.estado((x, y))]))
    while J.jugando:
        sleep(.5)
        cambios = False
        # Inicia busqueda casilla por casilla
        for i in range(J.x):
            for j in range(J.y):

                x, y = (i, j)
                # Analisa las casillas que aún no han sido marcadas.
                if J.estado((x, y)) == 11:
                    if P:
                        print(f"Inicia ({x},{y})")
                    # Pregunta si hay una mina confirmada, para poder marcarla.
                    if ASK(objetivo=J.desc.P([x, y, 9]), valor=True, base=B):
                        J.transicion((x,y),1)
                        cambios = True
                        if P:
                            print( f"Se encontró 💣 en ({x},{y})... tranqui, ya la marcamos!" )
                            clear_output(wait=True)
                            J.pintar_frente()
                            

                    # Pregunta si con certesa no hay bombas, para así poder
                    # destapar el cuadro.
                    elif ASK(objetivo=f"-{J.desc.P([x, y, 9])}", valor=True, base=B):
                        J.transicion((x,y))
                        B.TELL(J.desc.P([x, y, J.estado((x, y))]))
                        cambios = True
                        if P:
                            print( f"({x},{y}) Está libre de minas, se puede descubrir!" )
                            clear_output(wait=True)
                            J.pintar_frente()

        # ================= Recorio todos las casilla desconocidas.

        if J.objetivo():
            if P:
                print( f"Felicitaciones, ha ganado!!!" )
            return B

        # Buscar una solución aleatoria a falta de oportunidades!!!
        elif not cambios:
            x,y = (randint(0,J.x-1), randint(0,J.y-1) )

            while J.estado((x,y)) != 11:
                x,y = (randint(0,J.x-1), randint(0,J.y-1) )

            J.transicion((x,y))
            B.TELL(J.desc.P([x, y, J.estado((x, y))]))
            if P:
                print( f"A la suerte en ({x},{y})!!!" )
                clear_output(wait=True)
                J.pintar_frente()
    if P:
        clear_output(wait=True)
        J.pintar_frente()
    return B


# RUN RUN RUN

In [18]:
from time import time

# Tamaño grid
x, y = (8, 8)
# Cantidad de minas
m = 0

informacion = [
    "tiene 0 minas adyacentes en",
    "tiene 1 mina adyacente en",
    "tiene 2 minas adyacentes en",
    "tiene 3 minas adyacentes en",
    "tiene 4 minas adyacentes en",
    "tiene 5 minas adyacentes en",
    "tiene 6 minas adyacentes en",
    "tiene 7 minas adyacentes en",
    "tiene 8 minas adyacentes en",
    "tiene mina en",
    "ha sido marcada en",
    "es desconocida en",
]


info = len(informacion)

#filas = [i for i in range(x)]
#columnas = [j for j in range(y)]

D = Descriptor([x, y, info])

J = MineSweeper(x, y, m, D)

formulas = []

suma = 0

for i in range(1000):
    start = time()
    formulas += formulas_unicidad(J)
    end = time()
    suma += (end-start)
    
suma /= 1000 

print(len(formulas))

print(f'suma: {suma}')
print(f'sex: {end - start}')

"""
J = MineSweeper(x, y, m, D)
B = LPQuery(
    list(
        set(formulas_unicidad(J) + formulas_perim_no_minas(J) + formulas_perim_minas(J))
    )
)

GAME = GameOn(J, B)
"""

5760000
suma: 0.01355700707435608
sex: 0.012845754623413086


'\nJ = MineSweeper(x, y, m, D)\nB = LPQuery(\n    list(\n        set(formulas_unicidad(J) + formulas_perim_no_minas(J) + formulas_perim_minas(J))\n    )\n)\n\nGAME = GameOn(J, B)\n'